In [4]:
import os
import io
import json
import pandas as pd
import openai
from typing import List, Dict
from sklearn.preprocessing import MultiLabelBinarizer
from google.colab import auth
auth.authenticate_user()
from google.colab import userdata
import numpy as np

from gspread_dataframe import set_with_dataframe
import gspread
from google.auth import default
creds, _ = default()


In [54]:
gc = gspread.authorize(creds)

worksheet = gc.open('college-data').worksheet('College Rankings')
rows = worksheet.get_all_values()
df = pd.DataFrame.from_records(rows[1:], columns=rows[0])
print(df.head(3))


                              University         Location College Ranking  \
0     California Institute of Technology     Pasadena, CA               6   
1  University of California--Los Angeles  Los Angeles, CA              15   
2     University of California, Berkeley     Berkeley, CA              17   

  Business School Ranking In State Cost of Attendance  \
0                       6                       65898   
1                      16                       14208   
2                      17                       16832   

  Out of State Cost of Attendance Enrollment SAT Average High School GPA  \
0                           65898       1023                               
1                           46503      33040                               
2                           51032      33405                               

  Admission Rate Website Average Class Size Cost of Living Index  \
0                                                                  
1                    

In [55]:
schema = {
    "name": "college_data",
    "description": "College research",
    "parameters": {
        "type": "object",
        "properties": {
            "In State Cost of Attendance": {"type": "number"},
            "Out of State Cost of Attendance": {"type": "number"},
            "Enrollment": {"type": "integer"},
            "SAT": {"type": "integer"},
            "My Fit Score": {"type": "number"},
            "Average High School GPA": {"type": "number"},
            "Admission Rate": {"type": "number"},
            "Website": {"type": "string"},
            "Average Class Size": {"type": "integer"},
            "Cost of Living Index": {"type": "number"},
            "Marketing Majors Available": {"type": "array", "items": {"type": "string"}},
            "Cheerleading Team": {"type": "boolean"}
        },
        "required": [
            "In State Cost of Attendance",
            "Out of State Cost of Attendance",
            "Enrollment",
            "SAT",
            "My Fit Score",
            "Average High School GPA",
            "Admission Rate",
            "Website",
            "Average Class Size",
            "Cost of Living Index",
            "Marketing Majors Available",
            "Cheerleading Team"
        ]
    }
}


In [56]:
def call_openai_api(college):
    """Calls the OpenAI API with function calling to extract structured data."""
    client = openai.OpenAI(api_key=userdata.get("OPENAI_API_KEY"))

    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {"role": "system", "content": "I need help extracting this data. Please only fill in any missing data you can but only include data that you know is correct."},
            {"role": "user", "content": college}
        ],
        functions=[schema],
        function_call={"name": "college_data"},
        temperature=0
    )

    return json.loads(response.choices[0].message.function_call.arguments)


In [59]:
#new_rows = []
for index, row in df[['University', 'Location', 'College Ranking', 'Business School Ranking']][247:].iterrows():
    try:
        row_dict = row.to_dict()
        input = json.dumps(row_dict)
        json_data = call_openai_api(input)
        new_row = row.to_dict()  # Convert the row to a dictionary

        if isinstance(json_data, dict):  # Ensure it's a dictionary
            for key, value in json_data.items():
                new_row[key] = value  # Create a new column for each JSON key

        new_rows.append(new_row)

    except Exception as e:
        print(f"Error processing row {index}: {e}")
        # Handle the error as needed, e.g., skip the row or log the error

new_df = pd.DataFrame(new_rows)
print(new_df.head())


                              University         Location College Ranking  \
0     California Institute of Technology     Pasadena, CA               6   
1  University of California--Los Angeles  Los Angeles, CA              15   
2     University of California, Berkeley     Berkeley, CA              17   
3    University of California, San Diego     La Jolla, CA              29   
4        University of California, Davis        Davis, CA              33   

  Business School Ranking  In State Cost of Attendance  \
0                       6                        79400   
1                      16                        35000   
2                      17                        39600   
3                      29                        35000   
4                      34                        37644   

   Out of State Cost of Attendance  Enrollment     SAT  My Fit Score  \
0                            79400         948  1560.0           9.5   
1                            65000       450

,0,1,2,3
0,University,Location,College Ranking,Business School Ranking


In [71]:
numeric_cols = ['SAT', 'Average High School GPA',
                'In State Cost of Attendance',
                'Out of State Cost of Attendance', 'Admission Rate', 'Average Class Size',
                'Cost of Living Index', 'College Ranking', 'Business School Ranking']

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')


In [72]:
creds, _ = default()
gc = gspread.authorize(creds)

# Open the Google Sheet
sh = gc.open('college-data')

# Select a worksheet (or create a new one)
try:
  worksheet = sh.worksheet('Extracted_Data') # Try to open existing sheet
except gspread.exceptions.WorksheetNotFound:
  worksheet = sh.add_worksheet(title='Extracted_Data', rows=100, cols=20) # Create a new sheet if not found

set_with_dataframe(worksheet, new_df)

In [82]:
worksheet = gc.open('college-data').worksheet('Extracted_Data')
rows = worksheet.get_all_values()
df = pd.DataFrame.from_records(rows[1:], columns=rows[0])

numeric_cols = ['SAT', 'Average High School GPA','Enrollment',
                'In State Cost of Attendance',
                'Out of State Cost of Attendance', 'Admission Rate', 'Average Class Size',
                'Cost of Living Index', 'College Ranking', 'Business School Ranking']

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df.dtypes

,0
University,object
Location,object
College Ranking,float64
Business School Ranking,float64
In State Cost of Attendance,float64
Out of State Cost of Attendance,float64
Enrollment,float64
SAT,float64
My Fit Score,object
Average High School GPA,float64


In [78]:
def add_ranking_scores(df):
    """
    Add component scores and final ranking to a DataFrame of schools.

    Args:
        df: Pandas DataFrame containing school data with revised field names

    Returns:
        DataFrame with added score columns
    """
    # Create a copy to avoid modifying the original
    result_df = df.copy()

    # Initialize new columns
    result_df['Academic_Fit_Score'] = np.nan
    result_df['Affordability_Score'] = np.nan
    result_df['Location_Bonus'] = np.nan
    result_df['Program_Value_Score'] = np.nan
    result_df['Final_Rank_Score'] = np.nan

    # Process each row
    for idx, row in df.iterrows():
        # Convert row to dictionary for compatibility with existing functions
        school = row.to_dict()

        # Calculate component scores
        academic_fit = calculate_academic_fit(school)
        affordability = calculate_affordability(school)
        location_bonus = calculate_location_value(school)
        program_value = calculate_program_value(school)

        # Calculate final rank with UPDATED WEIGHTS
        final_score = (
            (academic_fit * 0.30) +     # 30% weight for academic fit
            (affordability * 0.30) +    # 30% weight for affordability
            (program_value * 0.40)      # 40% weight for program value
        ) * location_bonus

        # Store scores in DataFrame
        result_df.at[idx, 'Academic_Fit_Score'] = academic_fit
        result_df.at[idx, 'Affordability_Score'] = affordability
        result_df.at[idx, 'Location_Bonus'] = location_bonus
        result_df.at[idx, 'Program_Value_Score'] = program_value
        result_df.at[idx, 'Final_Rank_Score'] = final_score

    # Sort DataFrame by final score (descending)
    result_df = result_df.sort_values('Final_Rank_Score', ascending=False)

    # Add rank column (1, 2, 3, etc.)
    result_df['Rank'] = range(1, len(result_df) + 1)

    return result_df


def calculate_academic_fit(school):
    """
    Calculate how well the school matches student's academic profile
    Optimized for schools where student is at bottom edge of admission range

    Updated for revised field names
    """
    student_gpa = 3.2
    student_sat = 1150
    components = []

    # GPA match - favors being at edge of admission range
    if "Average High School GPA" in school and pd.notna(school["Average High School GPA"]):
        school_gpa = school["Average High School GPA"]
        gpa_diff = school_gpa - student_gpa

        # Scoring system that favors being at bottom edge of range
        if gpa_diff <= 0:
            # Below average - less desirable
            gpa_score = 75 - abs(gpa_diff * 20)
        elif gpa_diff <= 0.3:
            # Within 0.3 points above - ideal "edge of range" zone
            gpa_score = 100 - (gpa_diff / 0.3) * 15
        else:
            # More than 0.3 points above - progressively less optimal
            gpa_score = 85 - (gpa_diff - 0.3) * 30

        components.append(max(0, min(100, gpa_score)))

    # SAT score match - favors being at edge of admission range
    # Updated for combined SAT score field
    if "SAT" in school and pd.notna(school["SAT"]):
        school_sat = school["SAT"]
        sat_diff = school_sat - student_sat

        # Scoring system that favors being at bottom edge of range
        if sat_diff <= 0:
            # Below average - less desirable
            sat_score = 75 - abs(sat_diff * 0.1)
        elif sat_diff <= 100:
            # Within 100 points above - ideal "edge of range" zone
            sat_score = 100 - (sat_diff / 100) * 15
        else:
            # More than 100 points above - progressively less optimal
            sat_score = 85 - (sat_diff - 100) * 0.1

        components.append(max(0, min(100, sat_score)))

    # Admission rate (focusing on schools with 30-50% acceptance rates)
    if "Admission Rate" in school and pd.notna(school["Admission Rate"]):
        # Convert decimal to percentage if needed
        rate = school["Admission Rate"] * 100 if school["Admission Rate"] <= 1 else school["Admission Rate"]

        # Non-linear scaling focusing on schools with 30-50% acceptance rates
        admission_score = 0
        if rate < 15:
            admission_score = rate * 1.5  # Reach schools, less favorable
        elif rate < 30:
            admission_score = 22.5 + (rate - 15) * 2.5  # Transition zone
        elif rate < 50:
            admission_score = 60 + (rate - 30) * 1.5  # Sweet spot
        elif rate < 80:
            admission_score = 90 - (rate - 50) * 0.5  # Less optimal
        else:
            admission_score = 75 - (rate - 80) * 0.3  # Safety schools
        components.append(admission_score)

    # Consider enrollment size (smaller schools may offer better support)
    if "Enrollment" in school and pd.notna(school["Enrollment"]):
        enrollment = school["Enrollment"]

        # Scoring based on enrollment size
        enrollment_score = 0
        if enrollment < 5000:
            enrollment_score = 90  # Small schools (high personal attention)
        elif enrollment < 15000:
            enrollment_score = 90 - ((enrollment - 5000) / 10000) * 10  # Medium schools
        elif enrollment < 30000:
            enrollment_score = 80 - ((enrollment - 15000) / 15000) * 15  # Large schools
        else:
            enrollment_score = 65 - min(15, ((enrollment - 30000) / 10000) * 5)  # Very large schools
        components.append(enrollment_score)

    # Return average of available components or default value
    return sum(components) / len(components) if components else 50


def calculate_affordability(school):
    """
    Calculate affordability score (higher = more affordable)
    Updated for revised field names
    """
    # Start with a default middle value
    cost_score = 50

    # California residents: check for in-state tuition first
    is_ca_school = is_california_school(school)

    if is_ca_school and "In State Cost of Attendance" in school and pd.notna(school["In State Cost of Attendance"]):
        # In-state tuition at California school
        cost = school["In State Cost of Attendance"]
        cost_score = 100 - (cost / 350)  # Scale: $35k = 0, $0 = 100

    elif not is_ca_school and "Out of State Cost of Attendance" in school and pd.notna(school["Out of State Cost of Attendance"]):
        # Out-of-state tuition for non-CA school
        cost = school["Out of State Cost of Attendance"]
        cost_score = 100 - (cost / 300)  # Scale: $30k = 0, $0 = 100

    # Adjust for cost of living if available
    if "Cost of Living Index" in school and pd.notna(school["Cost of Living Index"]):
        col_index = school["Cost of Living Index"]
        # California average COL index is around 140-150
        # Adjust score based on deviation from 100 (national average)
        cost_score = cost_score * (100 / col_index)

    return max(0, min(100, cost_score))


def calculate_location_value(school):
    """
    Calculate location value (multiplier that advantages CA schools and penalizes
    out-of-state public schools without reciprocity)

    Updated for revised field names
    """
    # Check if California state school (UC or CSU system)
    if is_california_state_school(school):
        return 1.35  # 35% boost

    # Check if other California school
    elif is_california_school(school):
        # Private schools in California
        if not is_public_school(school):
            return 1.20  # 20% boost for private CA schools
        return 1.25  # 25% boost for other public CA schools

    # Check for schools with California resident benefits (reciprocity)
    elif has_california_resident_benefits(school):
        return 1.15  # 15% boost

    # Check for private schools in nearby states (AZ, NV, OR)
    elif is_nearby_state_school(school) and not is_public_school(school):
        return 1.10  # 10% boost for private schools in neighboring states

    # Penalty for out-of-state public schools without reciprocity
    elif is_public_school(school) and not has_california_resident_benefits(school):
        return 0.85  # 15% penalty

    # Standard multiplier for other schools
    return 1.0


def calculate_program_value(school):
    """
    Calculate program quality/fit - now weighted 40%
    Updated for revised field names
    """
    components = []

    # Check for marketing major availability
    if "Marketing Majors Available" in school and isinstance(school["Marketing Majors Available"], list):
        has_marketing = len(school["Marketing Majors Available"]) > 0
        components.append(90 if has_marketing else 40)

    # Consider business school ranking if available
    if "Business School Ranking" in school and pd.notna(school["Business School Ranking"]):
        ranking = school["Business School Ranking"]
        # Non-linear scaling that favors top 50 programs
        rank_score = 0
        if ranking <= 10:
            rank_score = 100 - (ranking - 1) * 2
        elif ranking <= 25:
            rank_score = 82 - (ranking - 10) * 1.2
        elif ranking <= 50:
            rank_score = 64 - (ranking - 25) * 0.6
        elif ranking <= 100:
            rank_score = 49 - (ranking - 50) * 0.3
        else:
            rank_score = 34 - min(34, (ranking - 100) * 0.1)
        components.append(rank_score)

    # Consider average class size if available
    if "Average Class Size" in school and pd.notna(school["Average Class Size"]):
        class_size = school["Average Class Size"]
        # Smaller classes are better (up to a point)
        size_score = 0
        if class_size <= 15:
            size_score = 100
        elif class_size <= 30:
            size_score = 100 - (class_size - 15) * 2
        elif class_size <= 60:
            size_score = 70 - (class_size - 30) * 1
        elif class_size <= 100:
            size_score = 40 - (class_size - 60) * 0.5
        else:
            size_score = 20 - min(20, (class_size - 100) * 0.1)
        components.append(size_score)

    # College general ranking if available
    if "College Ranking" in school and pd.notna(school["College Ranking"]):
        ranking = school["College Ranking"]
        # Non-linear scaling that favors top 100 programs
        rank_score = 0
        if ranking <= 25:
            rank_score = 100 - (ranking - 1) * 0.8
        elif ranking <= 50:
            rank_score = 80 - (ranking - 25) * 0.6
        elif ranking <= 100:
            rank_score = 65 - (ranking - 50) * 0.3
        else:
            rank_score = 50 - min(30, (ranking - 100) * 0.1)
        components.append(rank_score)

    # Marketing-specific or specialized business programs
    if "Marketing Majors Available" in school and isinstance(school["Marketing Majors Available"], list):
        special_programs = [
            p for p in school["Marketing Majors Available"]
            if any(term in str(p).lower() for term in ["digital", "analytics", "media", "international"])
        ]
        if special_programs:
            components.append(85 + min(15, len(special_programs) * 5))

    # Consider cheerleading team as a potential bonus
    if "Cheerleading Team" in school and pd.notna(school["Cheerleading Team"]):
        has_cheerleading = bool(school["Cheerleading Team"])
        if has_cheerleading:
            components.append(55)  # Modest bonus for having a cheerleading team

    # Return average of available components or default
    return sum(components) / len(components) if components else 50


def is_california_school(school):
    """Helper function to check if school is in California"""
    if "Location" not in school or pd.isna(school["Location"]):
        return False

    location = str(school["Location"]).lower()
    return ("california" in location or
            ", ca" in location or
            location.endswith(" ca"))


def is_california_state_school(school):
    """
    Helper function to check if school is a California state school
    Updated for "University" field name
    """
    if "University" not in school or pd.isna(school["University"]):
        return False

    name = str(school["University"]).lower()
    return ("california state university" in name or
            "cal state" in name or
            "csu " in name or
            "university of california" in name or
            ("uc " in name and "university of cincinnati" not in name))


def has_california_resident_benefits(school):
    """Helper function to check for California resident benefits (reciprocity)"""
    # Check for significant in-state vs out-of-state difference
    if ("In State Cost of Attendance" in school and
        "Out of State Cost of Attendance" in school and
        pd.notna(school["In State Cost of Attendance"]) and
        pd.notna(school["Out of State Cost of Attendance"])):

        in_state = school["In State Cost of Attendance"]
        out_state = school["Out of State Cost of Attendance"]

        # If there's at least 30% discount for in-state
        if (out_state - in_state) / out_state >= 0.3:
            return True

    # Check for WUE (Western Undergraduate Exchange) schools
    # This is a simplified check based on location
    if is_nearby_state_school(school) and is_public_school(school):
        return True

    return False


def is_nearby_state_school(school):
    """Check if school is in AZ, NV, OR, WA, or other neighboring states"""
    if "Location" not in school or pd.isna(school["Location"]):
        return False

    location = str(school["Location"]).lower()
    nearby_states = ["arizona", ", az", " az",
                    "nevada", ", nv", " nv",
                    "oregon", ", or", " or",
                    "washington", ", wa", " wa",
                    "utah", ", ut", " ut",
                    "idaho", ", id", " id"]

    return any(state in location for state in nearby_states)


def is_public_school(school):
    """
    Determine if a school is public based on name patterns
    Updated for "University" field name
    """
    if "University" not in school or pd.isna(school["University"]):
        return False

    name = str(school["University"]).lower()
    public_indicators = [
        "university of", "state university", "state college",
        "polytechnic", "a&m", "community college"
    ]

    private_indicators = [
        "college of", "institute of technology", "seminary"
    ]

    # Check for definite public indicators
    if any(indicator in name for indicator in public_indicators):
        return True

    # Check for definite private indicators
    if any(indicator in name for indicator in private_indicators):
        return False

    # Look for state names in the school name as a fallback
    state_names = ["alabama", "alaska", "arizona", "arkansas", "california",
                  "colorado", "connecticut", "delaware", "florida", "georgia",
                  "hawaii", "idaho", "illinois", "indiana", "iowa", "kansas",
                  "kentucky", "louisiana", "maine", "maryland", "massachusetts",
                  "michigan", "minnesota", "mississippi", "missouri", "montana",
                  "nebraska", "nevada", "new hampshire", "new jersey", "new mexico",
                  "new york", "north carolina", "north dakota", "ohio", "oklahoma",
                  "oregon", "pennsylvania", "rhode island", "south carolina",
                  "south dakota", "tennessee", "texas", "utah", "vermont",
                  "virginia", "washington", "west virginia", "wisconsin", "wyoming"]

    return any(f"{state} " in f" {name} " for state in state_names)

In [88]:
ranked_df = add_ranking_scores(df)
ranked_df.columns

Index(['University', 'Location', 'College Ranking', 'Business School Ranking',
       'In State Cost of Attendance', 'Out of State Cost of Attendance',
       'Enrollment', 'SAT', 'My Fit Score', 'Average High School GPA',
       'Admission Rate', 'Website', 'Average Class Size',
       'Cost of Living Index', 'Marketing Majors Available',
       'Cheerleading Team', 'Academic_Fit_Score', 'Affordability_Score',
       'Location_Bonus', 'Program_Value_Score', 'Final_Rank_Score', 'Rank'],
      dtype='object')

In [90]:
new_order = ['Rank', 'University', 'Location', 'Website', 'SAT',
 'Average High School GPA', 'Marketing Majors Available', 'Cheerleading Team',
 'Average Class Size', 'College Ranking', 'Business School Ranking', 'Academic_Fit_Score',
 'Affordability_Score', 'Location_Bonus', 'Program_Value_Score', 'Final_Rank_Score', 'In State Cost of Attendance',
 'Out of State Cost of Attendance', 'Admission Rate', 'Cost of Living Index']


ranked_df = ranked_df[new_order]


In [91]:
try:
  worksheet = sh.worksheet('Ranked_Data') # Try to open existing sheet
except gspread.exceptions.WorksheetNotFound:
  worksheet = sh.add_worksheet(title='Ranked Data', rows=100, cols=20) # Create a new sheet if not found

set_with_dataframe(worksheet, ranked_df )